In [133]:
import numpy as np
import pandas as pd

In [134]:
df = pd.read_pickle("jobs_df.pkl")
df

,title,link,date_of_scraping
0,Python Developer,https://jobs.techloop.io/job/23861,2023-05-06
1,Python Developer,https://jobs.techloop.io/job/21664,2023-05-06
2,SW VÝVOJÁŘ (CMAKE/PYTHON),https://jobs.techloop.io/job/11103,2023-05-06
3,Vývojář automatických testů (Embedded software...,https://jobs.techloop.io/job/23345,2023-05-06
4,Senior Electronic Design Automation Engineer (...,https://jobs.techloop.io/job/8537,2023-05-06
...,...,...,...
70,DevOps Engineer – Artifacts Repositories - Brno,https://jobs.techloop.io/job/22058,2023-05-06
71,Artificial Intelligence Intern (part-time),https://jobs.techloop.io/job/23383,2023-05-06
72,Datový analytik/Datová analytička (Bioinformatik),https://jobs.techloop.io/job/23235,2023-05-06
73,Senior Site Reliability Engineer,https://jobs.techloop.io/job/22214,2023-05-06


In [135]:
dff = df.copy()
dff.loc[0, 'link'] = 'aaaaaaaaaaa'
dff.loc[0, 'date_of_scraping'] = '2023-05-08'
dff.loc[1, 'link'] = 'aaaaaaaaaaa'
dff.loc[1, 'date_of_scraping'] = '2023-05-09'
dff

,title,link,date_of_scraping
0,Python Developer,aaaaaaaaaaa,2023-05-08
1,Python Developer,aaaaaaaaaaa,2023-05-09
2,SW VÝVOJÁŘ (CMAKE/PYTHON),https://jobs.techloop.io/job/11103,2023-05-06
3,Vývojář automatických testů (Embedded software...,https://jobs.techloop.io/job/23345,2023-05-06
4,Senior Electronic Design Automation Engineer (...,https://jobs.techloop.io/job/8537,2023-05-06
...,...,...,...
70,DevOps Engineer – Artifacts Repositories - Brno,https://jobs.techloop.io/job/22058,2023-05-06
71,Artificial Intelligence Intern (part-time),https://jobs.techloop.io/job/23383,2023-05-06
72,Datový analytik/Datová analytička (Bioinformatik),https://jobs.techloop.io/job/23235,2023-05-06
73,Senior Site Reliability Engineer,https://jobs.techloop.io/job/22214,2023-05-06


In [136]:
df= pd.merge(df,dff, how='outer')
df['date_of_scraping'] = df['date_of_scraping'].astype('datetime64[D]')
df.sort_values(by=['date_of_scraping'], ascending=False, ignore_index=True, inplace=True)
df.drop_duplicates('link', keep='last', inplace=True)
df = df.reset_index(drop=True)
df

,title,link,date_of_scraping
0,Python Developer,aaaaaaaaaaa,2023-05-08
1,Linux Specialist,https://jobs.techloop.io/job/22799,2023-05-06
2,"Business Process Analyst, Senior",https://jobs.techloop.io/job/21973,2023-05-06
3,Business Application Developer pro AI,https://jobs.techloop.io/job/21104,2023-05-06
4,VÝVOJÁŘ APLIKACÍ - vítáme čerstvé ABSOLVENTY I...,https://jobs.techloop.io/job/7919,2023-05-06
...,...,...,...
71,IT specialist / System Administrator,https://jobs.techloop.io/job/24015,2023-05-06
72,Software Test Engineer (m/ž),https://jobs.techloop.io/job/17900,2023-05-06
73,Specialista/ka datové analytiky - Data scientist,https://jobs.techloop.io/job/22651,2023-05-06
74,TECHNICAL WRITER,https://jobs.techloop.io/job/23239,2023-05-06


In [148]:
df.loc[2, 'link'] = df.apply(lambda x: ['a','asbs'], axis=1)
df.loc[2, 'link'] = ('abs', 'absss')

ValueError: Incompatible indexer with Series

In [143]:
for x in df.iterrows():
    print(x[0])
    print(x[1])
    print(x[1].title)
    print(type(x[1]))
    break

0
title                  Python Developer
link                        aaaaaaaaaaa
date_of_scraping    2023-05-08 00:00:00
Name: 0, dtype: object
Python Developer
<class 'pandas.core.series.Series'>


In [138]:
text = """<p class="MuiTypography-root MuiTypography-body1 css-1rlo451"><span>About “Data Strategy, Management &amp; Ethics” team
<p>The team is based in Prague and Brno and focuses on strategic consulting services in the topic of data. We design data strategies for global clients, build data implementation roadmaps, set overall data visions, help manage data (e.g. data governance, quality, MDM etc.), advise on the use of advanced analytics (e.g. ML, NLP etc.) and drive regulatory and ethical compliance (e.g. EU’s AI Act etc.). Being part of a wider ‘Digital Enablement’ area, we work closely with other teams focused on Big Data &amp; Data Science, Business Intelligence &amp; Reporting, as well as Digital Transformation and Software Development.</p>
About Manager
<p>Patrik is the team manager leading consulting services and being responsible for the Data Strategy, Management &amp; Ethics team. Patrik studied at University of Kent and dedicated his early career to development of analytics and business intelligence before transitioning to advisory roles. In recent years, Patrik worked with some of Europe's largest organisations: supported an Energy client in the Netherlands; advised a major Retailer in Sweden; and built a strategy for a smart city in Saudi Arabia. Patrik enjoys free time with his family and stays active trail running, and walking his dog Molly.</p>
Job description &amp; summary
<p>As a part of our “Data Strategy, Management &amp; Ethics” team you will:</p>
<ul>
<li>Develop custom Python code and solutions.</li>
<li>Continuously identify new solution concepts and innovations.</li>
<li>Develop and support already running solutions and tools both in Czechia and globally for internal and external clients.</li>
<li>Collaborate and interact with cross-functional project, business and development teams.</li>
<li>Operate in a focused team that will require a proactive and independent attitude.</li>
<li>Help shape and expand the current practice by introducing new technologies and approaches.</li>
<li>Share best practices and knowledge both internally and externally.</li>
<li>Requirement of the role</li>
<li>Minimum of 1-2 years of experience of development in Python.</li>
<li>Python programming skills and knowledge of Git.</li>
<li>Experience preferred with standard libraries like numpy, pandas, scikit-learn, requests, openpyxl, pyodbc and similar.</li>
<li>Open-minded and creative to try out new frameworks, tools and technologies.</li>
<li>Proactive in development of new concepts and solutions.</li>
<li>Able to work independently and interact with stakeholders across hierarchies.</li>
<li>Professionally fluent in English and preferred but not essential Czech or Slovak.</li>
</ul>
What we are offering
<ul>
<li>Stable and challenging working environment providing plenty of opportunities for your continuous professional growth.</li>
<li>Paid time off 25 days, 3 sick days and 1-2 extra days off at the end of the year.</li>
<li>High-end Ultrabook and iPhone with unlimited data.</li>
<li>Benefit program with 30.000 points that you can use for holidays, education, food vouchers, sports (Multisport), health... Simply on what you enjoy.</li>
<li>Big support for your education and development. Each employee has their own budget and your proactivity is highly welcomed.</li>
<li>After-work and team activities (team building, ski weekend, etc..)</li>
<li>You will develop your skills while having the support from the top notch professionals in the team (e.g. career coaching, etc).</li>
<li>Opportunity to work in the future at a foreign PwC office, for example in the USA, Canada and there are others in the selection.</li>
</ul>
<p>PwC is the largest audit team in the Czech Republic, law, tax, consulting and technology, data or forensic teams. Find out how easy it is to combine this knowledge when you're in the right place.</p>
<p>With us, you will get the opportunity to see how business is done in large companies. We are part of an international network of companies with more than 327,000 colleagues in 155 countries.</p>
<p>Once you've applied for a position, our recruiter for the Digital Enablement team will contact you . You'll then have an interview/selection process. If you're successful and you also get along with the hiring manager, you're good to go and start your career!</p></span></p>"""

In [139]:
text = str.lower(text)

In [140]:
possible_skills = ['python', 'git', 'pandas', 'scikit-learn', 'jenkins', 'numpy', 'bash', 'linux', 'sql']
req_skills = [skill for skill in possible_skills if skill in text]
req_skills

['python', 'git', 'pandas', 'scikit-learn', 'numpy']